In [7]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
from matplotlib import cycler
import statsmodels.api as sm
import statsmodels.stats.api as sms
import getFamaFrenchFactors as gff
import yfinance as yf
from datetime import datetime, date
import math 
from statsmodels.tsa.stattools import adfuller
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

pd.options.mode.chained_assignment = None 

In [630]:
api = '123456789'

In [631]:
stock_one = 'AAPL'
ticker_list = ['MSFT']

In [632]:
def growth(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    
    return data

def ratios(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    

    return data


def key(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    

    return data


def quote(ticker, api_key, as_reported=False):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?serietype=line&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?serietype=line&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}

    return data

In [633]:
df_initial1 = growth(stock_one, api, as_reported=False, limit=130)
df_initial1 = pd.json_normalize(df_initial1)

df_initial2 = key(stock_one, api, as_reported=False, limit=130)
df_initial2 = pd.json_normalize(df_initial2)
    
for stock in ticker_list:
    df_new1 = growth(stock, api, as_reported=False, limit=130)
    df_new1 = pd.json_normalize(df_new1)
    df_initial1 = pd.concat([df_new1,df_initial1],axis = 0)
    
for stock in ticker_list:
    df_new2 = key(stock, api, as_reported=False, limit=130)
    df_new2 = pd.json_normalize(df_new2)
    df_initial2 = pd.concat([df_new2,df_initial2],axis = 0)

In [634]:
df_initial1['name'] = df_initial1['symbol'] + df_initial1['date']
df_initial1['date'] = pd.to_datetime(df_initial1['date'])

df_initial2['name'] = df_initial2['symbol'] + df_initial2['date']
df_initial2['date'] = pd.to_datetime(df_initial2['date'])

df_initial3 = pd.merge(df_initial1, df_initial2, left_on='name', right_on='name')

df_initial3 = df_initial3.rename(columns={"date": "date_x"})
df_initial3 = df_initial3.rename(columns={"symbol": "symbol_x"})
df_initial3 = df_initial3.rename(columns={"period": "period_x"})

In [635]:
FIELDS = ['close','date']

df_initial2 = quote(stock_one, api, as_reported=False)
df_initial2 = pd.json_normalize(df_initial2['historical'])
df_initial2['symbol_x'] = stock_one

for stock in ticker_list:
    try:
        df_new2 = quote(stock, api, as_reported=False)
        df_new2 = pd.json_normalize(df_new2['historical'])
        df_new2 = df_new2[FIELDS]
        df_new2['symbol_x'] = stock 
        df_initial2 = pd.concat([df_new2,df_initial2],axis = 0)
    except:
        pass

In [636]:
df_initial2 = df_initial2.rename(columns={"date": "date_x"})
df_initial2 = df_initial2.loc[:, ['symbol_x','date_x','close']]

df_1 = df_initial2[(df_initial2['date_x']>='2002-01-01') & (df_initial2['date_x']<='2022-03-31')]
df_2 = df_initial3[(df_initial3['date_x']>='2002-01-01') & (df_initial3['date_x']<='2022-03-31')]

df_1['close'] = df_1.groupby('symbol_x')['close'].shift(-1)

df_1['date_x'] = pd.to_datetime(df_1['date_x'])
df_2['date_x'] = pd.to_datetime(df_2['date_x'])

df_2['year'] = pd.DatetimeIndex(df_2['date_x']).year
df_2['month'] = pd.DatetimeIndex(df_2['date_x']).month

df = pd.merge_asof(df_1.sort_values('date_x'),df_2.sort_values('date_x'), on='date_x', by='symbol_x', tolerance=pd.Timedelta("2 day"))

df1 = df.dropna(subset=['period_x']).sort_values(['symbol_x','date_x'])
df1 = df.dropna(subset=['period_x'])

df = df.drop_duplicates(subset=['currentRatio'], keep='first')

In [639]:
df.to_csv('/Users/daniil_bruk/Desktop/Main_US.csv', index = True, header=True)
df_2.to_csv('/Users/daniil_bruk/Desktop/Main_financials_US.csv', index = True, header=True)
df_1.to_csv('/Users/daniil_bruk/Desktop/Main_prices_US.csv', index = True, header=True)

## Data Preparation 2

In [8]:
df = pd.read_csv('Main_US.csv')
df_1 = pd.read_csv('Main_prices_US.csv')
df_2 = pd.read_csv('Main_financials_US.csv')

df_m = df_1.sort_values(by= ['symbol_x','date_x'],ascending = True)

df_m['date_x'] = pd.to_datetime(df_m['date_x'])
df_m = df_m.set_index(['date_x'])

df_m = df_m.groupby('symbol_x')['close'].resample('A').last().pct_change().dropna()

df_m = pd.DataFrame(df_m)
df_m = df_m.reset_index()

df_m['day'] = pd.DatetimeIndex(df_m['date_x']).day
df_m['month'] = pd.DatetimeIndex(df_m['date_x']).month

df2 = df_1

df2['day'] = pd.DatetimeIndex(df2['date_x']).day
df2['month'] = pd.DatetimeIndex(df2['date_x']).month
df2['year'] = pd.DatetimeIndex(df2['date_x']).year 

df_dec = df2.loc[(df2['month'] == 12)]
df_dec = df_dec.drop_duplicates(subset=['symbol_x','year'], keep='first')

df_dec = df_dec[['symbol_x','close','year']]

df = df.dropna(subset=['year'])

df['year'] = df['year'].astype('int64')

df_m_1 = df_m[df_m['month'] > 3]
df_m_1['year'] = pd.DatetimeIndex(df_m_1['date_x']).year - 1
df_m_2 = df_m[df_m['month'] <= 3]
df_m_2['year'] = pd.DatetimeIndex(df_m_2['date_x']).year - 2

df_12_1 = pd.merge_asof(df_m_1.sort_values('year'),df.sort_values('year'), on='year', by='symbol_x')
df_12_2 = pd.merge_asof(df_m_2.sort_values('year'),df.sort_values('year'), on='year', by='symbol_x')

df_12 = pd.concat([df_12_1, df_12_2], ignore_index=True)

df_12 = pd.merge_asof(df_12.sort_values('year'),df_dec.sort_values('year'), on='year', by='symbol_x')

df_12 = df_12.rename(columns={"close_x": "ret_m"})
df_12 = df_12.rename(columns={"close": "close_dec"})

## Final Data

In [9]:
m = df_12[['symbol_x',
'date_x_x',
'year',
'netIncomePerShare',
'bookValuePerShare',
'marketCap',
'ret_m',
'close_dec',
'threeYNetIncomeGrowthPerShare']]

m['month_ind'] = pd.DatetimeIndex(m['date_x_x']).month
m['year'] = pd.DatetimeIndex(m['date_x_x']).year 

m = m.rename(columns={"date_x_x": "date", "symbol_x": "ticker", "year": "year_ind"})
m = m.sort_values(by = ['ticker','date'])
m = m.dropna()

m = m[(m['date']>='2007-01-01') & (m['date']<='2022-03-31')]
m = m[(m['netIncomePerShare']> 0) & (m['bookValuePerShare']> 0)]
m = m[(m['ret_m']> -1) & (m['ret_m'] < 100)]
m = m[(m['threeYNetIncomeGrowthPerShare'] > 0) & (m['threeYNetIncomeGrowthPerShare'] < 4) ]

m['pe_dec'] = m['close_dec'] / m['netIncomePerShare']
m['peg'] = m['pe_dec'] / (m['threeYNetIncomeGrowthPerShare'] * 100)
m = m[(m['peg'] < 40)]

ff3_monthly = gff.famaFrench3Factor(frequency='m')
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)

ff3_monthly['year'] = pd.DatetimeIndex(ff3_monthly['Date']).year
ff3_monthly['month'] = pd.DatetimeIndex(ff3_monthly['Date']).month

ff3_monthly = ff3_monthly.sort_values(by = ['Date'], ascending=False)


m['year'] = pd.DatetimeIndex(m['date']).year
m['month'] = pd.DatetimeIndex(m['date']).month

m = m.merge(ff3_monthly, on=['year','month'])

m = m[['ticker',
'date',
'year_ind',
'month_ind',
'netIncomePerShare',
'threeYNetIncomeGrowthPerShare',
'marketCap',
'ret_m',
'pe_dec',
'peg',
'Mkt-RF',
'SMB',
'HML',
'RF']]

In [10]:
m.corr()

,year_ind,month_ind,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg,Mkt-RF,SMB,HML,RF
year_ind,1.000000,NaN,-0.019121,-0.011750,0.076958,0.086287,0.081497,0.046849,-0.078834,-0.297150,-0.023003,0.020480
month_ind,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
netIncomePerShare,-0.019121,NaN,1.000000,0.047258,-0.000042,-0.021060,-0.009140,-0.007772,0.005906,0.025508,-0.006894,-0.009422
threeYNetIncomeGrowthPerShare,-0.011750,NaN,0.047258,1.000000,-0.013469,-0.004136,-0.003478,-0.287190,-0.023925,0.021137,-0.035523,0.050626
marketCap,0.076958,NaN,-0.000042,-0.013469,1.000000,0.011882,0.003617,-0.010796,0.004610,-0.008860,-0.002622,-0.000365
ret_m,0.086287,NaN,-0.021060,-0.004136,0.011882,1.000000,-0.015768,-0.011656,0.207299,0.055024,0.133574,-0.054532
pe_dec,0.081497,NaN,-0.009140,-0.003478,0.003617,-0.015768,1.000000,0.257918,-0.044533,-0.065059,-0.020855,0.064960
peg,0.046849,NaN,-0.007772,-0.287190,-0.010796,-0.011656,0.257918,1.000000,-0.020718,-0.036287,0.007652,0.008658
Mkt-RF,-0.078834,NaN,0.005906,-0.023925,0.004610,0.207299,-0.044533,-0.020718,1.000000,0.509419,0.482402,-0.521197
SMB,-0.297150,NaN,0.025508,0.021137,-0.008860,0.055024,-0.065059,-0.036287,0.509419,1.000000,0.070254,-0.325945


## Descriptive statistics

In [11]:
ff = m

ff['peg'] = ff['peg'].astype('float64')
ff['pe_dec'] = ff['pe_dec'].astype('float64')
ff['netIncomePerShare'] = ff['netIncomePerShare'].astype('float64')
ff['threeYNetIncomeGrowthPerShare'] = ff['threeYNetIncomeGrowthPerShare'].astype('float64')
ff = ff[['netIncomePerShare','threeYNetIncomeGrowthPerShare','marketCap','ret_m','pe_dec','peg']]

gg = ff.describe()
gg.round(6)

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
count,8702.000000,8702.000000,8.702000e+03,8702.000000,8702.000000,8702.000000
mean,6.713647,0.815833,1.520831e+10,0.123206,22.874313,1.077382
std,200.368519,0.794376,5.675959e+10,0.480180,29.192017,2.850466
min,0.009013,0.002026,6.020660e+05,-0.965035,0.028233,0.000361
25%,1.187297,0.256958,8.867568e+08,-0.110204,13.528065,0.158993
50%,2.142547,0.550928,2.787883e+09,0.095542,18.774869,0.361778
75%,3.692472,1.095721,9.824353e+09,0.308053,25.689633,0.834421
max,13421.778963,3.997679,1.996361e+12,25.120000,1872.984821,39.535513


In [12]:
ff3_monthly_2 = ff3_monthly[(ff3_monthly['year'] > 2007)]
ff3_monthly_2 = ff3_monthly_2[['Mkt-RF','SMB','HML','RF']]*100
ff3_monthly_2.describe().iloc[:,[0,1,2,3]].round(5)

,Mkt-RF,SMB,HML,RF
count,172.00000,172.00000,172.00000,172.00000
mean,0.84843,0.08186,-0.09936,0.04215
std,4.73121,2.48532,3.31726,0.06480
min,-17.23000,-5.93000,-13.92000,0.00000
25%,-1.60000,-1.86000,-1.90250,0.00000
50%,1.39500,0.15500,-0.38000,0.01000
75%,3.45250,1.57000,1.48000,0.06250
max,13.65000,7.12000,12.74000,0.21000


## Raw return analysis

In [13]:
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
monthly = s.groupby(['month_ind','quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['month_ind','quantile']).mean()
    monthly = pd.concat([g,monthly])
    

df_mean = pd.DataFrame(monthly)
df_mean['ret_m'] = df_mean['ret_m'] * 100
 
df_mean = df_mean.reset_index()

In [14]:
df_mean.groupby(['quantile']).mean().round(5).iloc[:,[2,3,4,5,6,7]]

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
quantile,,,,,,
P1,21.38485,1.88874,1.075712e+10,15.38426,13.42542,0.07865
P2,3.07384,0.67236,1.717463e+10,12.93344,22.56908,0.43523
P3,2.36951,0.14327,1.240701e+10,11.34665,32.30044,3.99139


In [15]:
df_mean.groupby('quantile').std().round(5).iloc[:,[2,3,4,5,6,7]]

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
quantile,,,,,,
P1,43.63003,0.14455,5.629786e+09,26.95639,2.63199,0.01548
P2,1.00482,0.10847,6.553781e+09,19.02561,4.36704,0.10232
P3,0.63790,0.03721,3.854448e+09,18.33663,9.14004,0.81518


In [16]:
hist_returns = df_mean.groupby(['year_ind','quantile'])['ret_m'].mean()
hist_returns = hist_returns.reset_index()

hist_returns.to_csv('/Users/daniil_bruk/Desktop/hist_wp_us.csv', index = True, header=True)

hist_returns_3 = df_mean.groupby(['year_ind'])['Mkt-RF'].mean()
hist_returns_3 = hist_returns_3.reset_index()

In [17]:
sharpe = 12 * (df_mean.groupby(['quantile'])['ret_m'].mean() - df_mean.groupby(['quantile'])['RF'].mean()) / (df_mean.groupby(['quantile'])['ret_m'].std() * math.sqrt(12))
pd.DataFrame(sharpe)

,0
quantile,
P1,1.976928
P2,2.354771
P3,2.143477


In [18]:
sharpe_market = 12 * (df_mean['Mkt-RF'].mean()) / (df_mean['Mkt-RF'].std() * math.sqrt(12))
sharpe_market

1.081075759896651

In [19]:
m_squared = sharpe * (df_mean['Mkt-RF'].std() * math.sqrt(12)) + df_mean['RF'].mean()
pd.DataFrame(m_squared)

,0
quantile,
P1,0.245708
P2,0.292570
P3,0.266364


In [21]:
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['PEG_quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
s['PE_quantile'] = pd.qcut(s['pe_dec'], q=[0, .2, .8, 1],labels=bin_labels_5)

monthly = s.groupby(['PEG_quantile','PE_quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['PEG_quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    s['PE_quantile'] = pd.qcut(s['pe_dec'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['PEG_quantile','PE_quantile']).mean()
    monthly = pd.concat([g,monthly])

df_mean_2 = pd.DataFrame(monthly['ret_m'])

df_mean_2 = df_mean_2.groupby(['PEG_quantile','PE_quantile']).mean().round(5).unstack()
df_mean_2

ret_m                  
PE_quantile        P1       P2       P3
PEG_quantile                           
P1            0.17201  0.13295  0.14455
P2            0.14136  0.12668  0.13131
P3            0.12380  0.10093  0.12618

In [22]:
#for the graph
df_graph = df_mean.groupby(['year_ind','month_ind','quantile']).mean().round(5).iloc[:,[3]]
df_graph.to_csv('/Users/daniil_bruk/Desktop/graph_US.csv', index = True, header=True)

## Creating PEG - factor values

In [23]:
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
monthly = s.groupby(['month_ind','quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['month_ind','quantile']).mean()
    monthly = pd.concat([g,monthly])
    

df_mean = pd.DataFrame(monthly)
 
df_mean = df_mean.reset_index()

df_lowest = df_mean[(df_mean['quantile'] == 'P1')]
df_highest = df_mean[(df_mean['quantile'] == 'P3')]
df_middle = df_mean[(df_mean['quantile'] == 'P2')]

df_final = df_lowest
df_final['factor_UMO'] = df_lowest['ret_m'].values - df_highest['ret_m'].values - df_final['RF']
df_final['factor_U'] = df_lowest['ret_m'].values - df_final['RF']
df_final['factor_O'] = df_highest['ret_m'].values - df_final['RF']
df_final['factor_M'] = df_middle['ret_m'].values - df_final['RF']
df = df_final

In [24]:
df_lowest = df_mean[(df_mean['quantile'] == 'P1')]
df_highest = df_mean[(df_mean['quantile'] == 'P3')]
df_middle = df_mean[(df_mean['quantile'] == 'P2')]


stats.ttest_ind(df_final['factor_U'], df_final['factor_M'], equal_var=False)

Ttest_indResult(statistic=0.2875032571941778, pvalue=0.7760803617192046)

## Tests

In [25]:
#find p-values for all!!!

y = df['factor_UMO']
x = df[['Mkt-RF','SMB','HML']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit()

test_result = sms.het_breuschpagan(results.resid, results.model.exog)
test_result[1]

0.39882918851031

In [26]:
y1 = df['factor_U']
y2 = df['factor_M']
y3 = df['factor_O']
y4 = df['factor_UMO']

y = {'1':y1,'2':y2,'3':y3,'4':y4}

y = pd.DataFrame(y)
result = adfuller(y['4'])
result[1]

0.11966045082835952

## Regression Analysis

In [28]:
y = df['factor_O']
x = df[['Mkt-RF','SMB','HML']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit()#cov_type='HC1')
results.summary()  

/Users/daniil_bruk/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               factor_O   R-squared:                       0.229
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     1.090
Date:                Sun, 05 Jun 2022   Prob (F-statistic):              0.394
Time:                        20:22:12   Log-Likelihood:                 6.6206
No. Observations:                  15   AIC:                            -5.241
Df Residuals:                      11   BIC:                            -2.409
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0865      0.053      1.639      0.129      -0.030       0.203
Mkt-RF         2.3839      1.805      1.321      0.213      -1.588       6.356
SMB           -0.8370      2.219     -0.377      0.713      -5.722       4.048
HML            0.6576      2.780      0.237      0.817      -5.461       6.776
==============================================================================
Omnibus:                        5.732   Durbin-Watson:                   2.612
Prob(Omnibus):                  0.057   Jarque-Bera (JB):                2.859
Skew:                          -0.617   Prob(JB):                        0.239
Kurtosis:                       4.747   Cond. No.                         67.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
results.rsquared

0.22909409939749226

In [30]:
y = df['factor_UMO']
x = df[['Mkt-RF']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit()#cov_type='HC1')
results.summary()  

/Users/daniil_bruk/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=15
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             factor_UMO   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.245
Method:                 Least Squares   F-statistic:                     5.551
Date:                Sun, 05 Jun 2022   Prob (F-statistic):             0.0348
Time:                        20:22:18   Log-Likelihood:                 15.535
No. Observations:                  15   AIC:                            -27.07
Df Residuals:                      13   BIC:                            -25.65
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0221      0.025      0.886      0.392      -0.032       0.076
Mkt-RF         1.5854      0.673      2.356      0.035       0.132       3.039
==============================================================================
Omnibus:                        0.327   Durbin-Watson:                   2.216
Prob(Omnibus):                  0.849   Jarque-Bera (JB):                0.468
Skew:                          -0.092   Prob(JB):                        0.791
Kurtosis:                       2.155   Cond. No.                         28.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""